<h1> Machine Learning - Test 2</h1>

<h2> Auteurs </h2>

* Forestier Quentin
* Herzig Melvyn


<h3> Importations </h3>
La première partie consiste à importer PyEvolve de manière à pouvoir utiliser notre algorithme génétique puis d'importer GeoPy pour calculer la distance entre les villes en tenant compte de la forme de la terre.

In [120]:
# For GA algorithm
from pyevolve import GSimpleGA
from pyevolve import Selectors
from pyevolve import Statistics
from pyevolve import G1DList
from pyevolve import Mutators
from pyevolve import Crossovers
from pyevolve import Consts

import random

# For cites distances
from geopy.distance import geodesic as GD

PIL_SUPPORT = None

try:
   from PIL import Image, ImageDraw, ImageFont
   PIL_SUPPORT = True
except:
   PIL_SUPPORT = False

<h3> Matrice des distances </h3>
Dans cette première, nous créons la matrice de distance à partir des latitudes et longitudes des villes. La matrice de distance est importante car elle sera utilisée plus tard dans la fonction de "fitness".

Pour ce faire, nous zippons les listes de coordonnées pour obtenir un tuple (lat, long) par ville. Ensuite, nous créons la matrices de distance à partir de cette liste de tuples. 


In [121]:
# Cities latitudes
LAT = [16.47, 16.47, 20.09, 22.39, 25.23, 22.00, 20.47, 
       17.20, 16.30, 14.05, 16.53, 21.52, 19.41, 20.09]

# Cities longitudes
LON = [96.10, 94.44, 92.54, 93.37, 97.24, 96.05, 97.02, 
       96.29, 97.38, 98.12, 97.38, 95.59, 97.13, 94.55]

def compute_distances_matrix(coordinates):
    '''
    Compute a distance matrix based on list of (lat, lon).
    The cell [i, j] is the distance from the coordinate i to the coordinate j
    This is not an euclidean disance, but a distance that consider the earth spherical form.
    '''
    matrix={}
    for i, coordinate_src in enumerate(coordinates):
        for j, coordinate_dist in enumerate(coordinates):
            matrix[i, j] = GD(coordinate_src, coordinate_dist).km
    return matrix

distances_matrix = compute_distances_matrix(list(zip(LAT, LON)))

<h3> Fonction de fitness </h3>
Pour calculer la fitness d'un chromosome, nous calculons la distance totale du tour. 
Plus le tour est court meilleur est son "fitness".
Pour chaque index, nous calculons la distances au prochain index.
Ainsi pour l'index 0 nous obtenons la distance entre la ville à l'index 0 et la ville à l'index 1.
De la même manière, pour le dernier index, nous obtenons la distance jusqu'à la ville à l'index 0. 

In [122]:
def fitness_tour_length(chromosome_tour):
    global distances_matrix
    tour_length = 0.0
    nb_cities = len(chromosome_tour)
    for i in range(nb_cities):
        j = (i + 1) % nb_cities  
        tour_length += distances_matrix[chromosome_tour[i], chromosome_tour[j]]
    return tour_length
    

In [123]:
def genome_initializator(genome, **args):
   """ 
   Initialize a genome
   source: http://pyevolve.sourceforge.net/examples.html#example-12-the-travelling-salesman-problem-tsp
   """
   genome.clearList()
   lst = [i for i in range(genome.getListSize())]

   for i in range(genome.getListSize()):
      choice = random.choice(lst)
      lst.remove(choice)
      genome.append(choice)

In [124]:
def write_tour_to_img(coords, tour, img_file):
   """ The function to plot the graph """
   padding=2
   coords=[(10*x+padding,10*y+padding) for (x,y) in coords]
   maxx,maxy,minx,miny=0,0,0,0
   for x,y in coords:
      maxx=max(x,maxx)
      maxy=max(y,maxy)
      minx=min(x,minx)
      miny=min(y,miny)
   maxx+=padding
   maxy+=padding
   minx-=padding
   miny-=padding
   img=Image.new("RGB",(int(maxx) - int(minx),int(maxy) - int(miny)),color=(255,255,255))

   font=ImageFont.load_default()
   d=ImageDraw.Draw(img);
   num_cities=len(tour)
   for i in range(num_cities):
      j=(i+1)%num_cities
      city_i=tour[i]
      city_j=tour[j]
      x1,y1=coords[city_i]
      x2,y2=coords[city_j]
      d.line((int(x1 - minx),int(y1 - miny),int(x2 - minx),int(y2 - miny)),fill=(0,0,0))
      d.text((int(x1 - minx)+7,int(y1 - )-5),str(i),font=font,fill=(32,32,32))

   for x,y in coords:
      x,y=int(x),int(y)
      d.ellipse((x-2,y-2,x+2,y+2),outline=(0,0,0),fill=(196,196,196))
   del d
   img.save(img_file, "PNG")

   print("The plot was saved into the %s file." % (img_file,))

<h1> Execution </h1>

In [125]:
number_of_cities = len(LAT)

# Setting genome
genome = G1DList.G1DList(number_of_cities)
genome.setParams(rangemin=0, rangemax=number_of_cities - 1)

genome.evaluator.set(fitness_tour_length)
genome.mutator.set(Mutators.G1DListMutatorSwap)
genome.crossover.set(Crossovers.G1DListCrossoverOX)
genome.initializator.set(genome_initializator)

ga = GSimpleGA.GSimpleGA(genome)
ga.setGenerations(1000)
ga.setMinimax(Consts.minimaxType["minimize"])
ga.setCrossoverRate(1.0)
ga.setMutationRate(0.03)
ga.setPopulationSize(80)

ga.evolve(freq_stats=100)
best = ga.bestIndividual()

print(best)

if PIL_SUPPORT:
    write_tour_to_img(list(zip(LAT, LON)), best, "tsp_result.png")
else:
      print("No PIL detected, cannot plot the graph !")

Gen. 0 (0.00%): Max/Min/Avg Fitness(Raw)             [8107.32(8591.07)/5499.19(5049.20)/6756.10(6756.10)]
Gen. 100 (10.00%): Max/Min/Avg Fitness(Raw)             [4665.69(6442.46)/3765.74(3486.22)/3888.08(3888.08)]
Gen. 200 (20.00%): Max/Min/Avg Fitness(Raw)             [4608.19(5875.06)/3706.57(3486.22)/3840.15(3840.15)]
Gen. 300 (30.00%): Max/Min/Avg Fitness(Raw)             [4657.46(5860.34)/3726.29(3486.22)/3881.22(3881.22)]
Gen. 400 (40.00%): Max/Min/Avg Fitness(Raw)             [4590.10(5987.07)/3705.18(3486.22)/3825.08(3825.08)]
Gen. 500 (50.00%): Max/Min/Avg Fitness(Raw)             [4661.58(5900.72)/3731.11(3486.22)/3884.65(3884.65)]
Gen. 600 (60.00%): Max/Min/Avg Fitness(Raw)             [4526.43(6528.95)/3693.82(3486.22)/3772.02(3772.02)]
Gen. 700 (70.00%): Max/Min/Avg Fitness(Raw)             [4679.68(6632.07)/3781.69(3486.22)/3899.73(3899.73)]
Gen. 800 (80.00%): Max/Min/Avg Fitness(Raw)             [4681.20(6000.64)/3746.87(3486.22)/3901.00(3901.00)]
Gen. 900 (90.00%): Max